# Figures

This notebook generates figures for the paper. Run this notebook last.

In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

## Data Prep

Load fire polygon data.

In [ ]:
study_fires = gpd.read_file("../data/study_fires.geojson").to_crs("EPSG:5070")
candidate_fires = gpd.read_file("../data/candidate_fires.geojson").to_crs("EPSG:5070")

study_fires["ha"] = study_fires.geometry.area / 10_000
study_fires["forested_ha"] = study_fires["ha"] * (study_fires["percent_forest"] / 100)

candidate_fires["ha"] = candidate_fires.geometry.area / 10_000
candidate_fires["forested_ha"] = candidate_fires["ha"] * (
    candidate_fires["percent_forest"] / 100
)
candidate_fires["year"] = pd.to_datetime(candidate_fires["Ig_Date"], unit="ms").dt.year

Load and process the trends data from the `trends` notebook.

In [ ]:
owner_trends = pd.read_csv("../data/results/owner_trends.csv")
owner_group_trends = pd.read_csv("../data/results/owner_group_trends.csv")

# Remove the severity grouping
owner_trends = (
    owner_trends.drop(columns=["severity"])
    .groupby(["year", "owner"])
    .agg("sum")
    .reset_index()
    .assign(
        cumulative_analysis_area=lambda x: x.groupby("owner")["analysis_area"].cumsum()
    )
    .assign(
        cumulative_harvest_area=lambda x: x.groupby("owner")["harvest_area"].cumsum()
    )
)

owner_group_trends = (
    owner_group_trends.drop(columns=["severity"])
    .groupby(["year", "owner"])
    .agg("sum")
    .reset_index()
    .assign(
        cumulative_analysis_area=lambda x: x.groupby("owner")["analysis_area"].cumsum()
    )
    .assign(
        cumulative_harvest_area=lambda x: x.groupby("owner")["harvest_area"].cumsum()
    )
)

Convert the candidate fires to annual trends.

In [ ]:
candidate_fire_trends = (
    candidate_fires.groupby("year")
    .agg({"forested_ha": "sum", "ha": "sum"})
    .reset_index()
    .assign(cumulative_forested_ha=lambda x: x["forested_ha"].cumsum())
)

## Summary Data

Print some summary stats for the paper.

In [ ]:
print(f"Study fires: {study_fires.shape[0]}")
print(f"Study fire forested area: {study_fires.forested_ha.sum():,.0f} ha")
print(f"Total forested area: {candidate_fires.forested_ha.sum():,.0f} ha")
print(f"Total analysis area: {owner_trends.analysis_area.sum():,.0f} ha")
print(
    "Percent of burned forest sampled: "
    f"{owner_trends.analysis_area.sum() / candidate_fires.forested_ha.sum():.2%}"
)
print(
    "Percent of burned forest in study fires: "
    f"{study_fires.forested_ha.sum() / candidate_fires.forested_ha.sum():.2%}"
)

## Make Figures

### Area Burned by Year

Compare forested area burned for the study fires and the region.

In [ ]:
owner_group_analysis_area = owner_group_trends[
    owner_group_trends.owner.ne("Other")
].melt(
    id_vars=["year", "owner"],
    value_vars=["analysis_area", "cumulative_analysis_area"],
    var_name="area_type",
    value_name="area",
)

In [ ]:
fig = px.line(
    owner_group_analysis_area,
    x="year",
    y="area",
    color="owner",
    color_discrete_sequence=["#33a02c", "#1f78b4", "#606060"],
    facet_row="area_type",
    markers=True,
    template="plotly_white",
    width=600,
    height=500,
    line_shape="spline",
    facet_row_spacing=0.1,
)

# Make the "all" line dashed
fig.data[4].line.dash = "dot"
fig.data[5].line.dash = "dot"


fig.add_bar(
    x=candidate_fire_trends.year,
    y=candidate_fire_trends.cumulative_forested_ha,
    name="Total",
    marker_color="#d0d0d0",
    row=1,
    col=1,
)

fig.add_bar(
    x=candidate_fire_trends.year,
    y=candidate_fire_trends.forested_ha,
    name="Total",
    marker_color="#d0d0d0",
    row=2,
    col=1,
)

fig.update_yaxes(title=None, matches=None)
fig.update_xaxes(title=None, dtick=5, ticks="inside")

# Delete facet labels
for i in fig.layout.annotations:
    i.text = ""

fig.update_layout(
    showlegend=False,
    margin=dict(l=0, r=0, t=0, b=0),
    font=dict(size=20),
)

fig.write_image("../figures/area_burned_combined.png", scale=3)

### Area Harvested Trends

Compare area harvested by ownership.

In [ ]:
owner_group_harvest_area = owner_group_trends[
    owner_group_trends.owner.ne("Other")
].melt(
    id_vars=["year", "owner"],
    value_vars=["harvest_area", "cumulative_harvest_area"],
    var_name="area_type",
    value_name="area",
)

In [ ]:
fig = px.line(
    owner_group_harvest_area,
    x="year",
    y="area",
    color="owner",
    color_discrete_sequence=["#33a02c", "#1f78b4", "#606060"],
    facet_row="area_type",
    markers=True,
    template="plotly_white",
    width=600,
    height=400,
    line_shape="spline",
    facet_row_spacing=0.1,
)

# Make the "all" line dashed
fig.data[4].line.dash = "dot"
fig.data[5].line.dash = "dot"

fig.update_yaxes(title=None, matches=None)
fig.update_xaxes(title=None, dtick=5, ticks="inside", showgrid=False)

# Delete facet labels
for i in fig.layout.annotations:
    i.text = ""

fig.update_layout(
    showlegend=False,
    margin=dict(l=0, r=0, t=0, b=0),
    font=dict(size=20),
)

fig.write_image("../figures/area_harvested_combined.png", scale=3)